In [228]:
# If necessary, install packages
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
ERROR: No matching distribution found for os


In [229]:
!pip install gdown

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
# file IDs and table namesfor lobbying data files from Google Drive
lob_rpt = ("1CIq7XwaFTJmnUKdAvYyh53Et2XSJRX9d", "lob_rpt")
lob_lobbyist = ("1q1ZyLvUhsxsbPLdi_CmrNH0FqLoorr6p", "lob_lobbyist")
lob_lobbying = ("19Jef89spXbkExNpYA-s_e8GRii-0tReu", "lob_lobbying")
lob_issue = ("18-D_vW4dAHJlvFs4ARK4lgx6KdsAXWoq", "lob_issue")
lob_indus = ("1yldZYLcZTLIMQ5Xa-e-ecGgQGFWv5n4A", "lob_indus")
lob_bills = ("1qqQvNXtdhyI8KYcctpVUZ3FoIwvjWaG1", "lob_bills")
lob_agency = ("1CFi0Itpi_qg4X4wicg1FIg8S_FJfvCN0", "lob_agency")
CatCodeReference = ("1vfVA-v-lUMuykF9efl2oDN5DJK2t1PwA", "CatCodeReference")
CandidateIds = ("1k2lhAcuSQbn5z7NN3USPEbX_Rg5VV_Ky", "CandidateIds")
congress_bill_data_id = ("1-MnzV0iZ2FPYiuQgOmggjmXvxGmBzYV8", "congress_bill_data")

In [232]:
import pandas as pd
import gdown
import os

def load_csv(table, save_file=False, file_path=None):
    """
    Downloads a large CSV file from Google Drive into a Pandas DataFrame.
    
    Parameters:
        file_info (tuple): A tuple containing the Google Drive file ID and the table name.
        save_file (bool): If True, the file is saved. If False, it is deleted after loading.
        file_path (str, optional): The directory path where the file should be saved. Defaults to current directory.

    Returns:
        pd.DataFrame: The loaded Pandas DataFrame.
    """
    file_id, table_name = table
    
    # Set filename as the table name
    temp_file = f"{table_name}.csv"
    
    # If a file path is provided, join it with the filename
    if file_path:
        temp_file = os.path.join(file_path, temp_file)

    # Construct Google Drive download URL
    url = f"https://drive.google.com/uc?export=download&id={file_id}"

    # Download the file
    gdown.download(url, temp_file, quiet=False)

    # Load the CSV file into a Pandas DataFrame
    df = pd.read_csv(temp_file,header=None)

    # Remove the file if save_file is False
    if not save_file:
        os.remove(temp_file)

    return df

In [233]:
# Example usage:
df = load_csv(lob_rpt, save_file=False)  # Specify file_path if needed
print(df.head())

Downloading...
From: https://drive.google.com/uc?export=download&id=1CIq7XwaFTJmnUKdAvYyh53Et2XSJRX9d
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_rpt.csv
100%|██████████| 3.29k/3.29k [00:00<00:00, 476kB/s]

                                  0    1
0                   MID-YEAR REPORT    m
1                MID-YEAR AMENDMENT   ma
2            MID-YEAR (NO ACTIVITY)   mn
3  MID-YEAR AMENDMENT (NO ACTIVITY)  man
4              MID-YEAR TERMINATION   mt


## Data Cleaning

In [234]:
# Creating function to visualize key aspects of each dataframe. 
def glimpse(df):
    print(f'\nShape of dataframe is: {df.shape}\n')
    print(f'Dataframe has the following columns:\n{df.columns}\n')
    print(f'Dataframe has the following data types:\n{df.dtypes}\n')
    print(f'Overview of missing values:\n{df.isnull().sum()}\n')
    print(f'Missing values as percentage of total:\n{(df.isnull().sum()/len(df))*100}\n')
    print(f'First 5 Columns:{df.head()}\n')
    print(f'Last 5 Columns:{df.tail()}\n')

### Lobbying Data

In [235]:
df_lobbying = load_csv(lob_lobbying, save_file = False)
df_lobbying.columns = ['UniqID','Registrant_Raw','Registrant','IsLobbyingFirm','Client_raw','Client','UItorg','Amount','Catcode','Source','Self','IncludeNSFS','Use','Ind','Year','Type','Typelong','Affiliate']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=19Jef89spXbkExNpYA-s_e8GRii-0tReu
From (redirected): https://drive.google.com/uc?export=download&id=19Jef89spXbkExNpYA-s_e8GRii-0tReu&confirm=t&uuid=1fd98d57-2087-4d98-bd65-9ad9bea70177
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_lobbying.csv
100%|██████████| 351M/351M [00:27<00:00, 12.6MB/s] 
/var/folders/pg/ghgx2xgn3qb65km4qhtfgzhr0000gn/T/ipykernel_96499/2418707545.py:33: DtypeWarning: Columns (7,11,14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(temp_file,header=None)


In [237]:
#Dropping unecessary columns
df_lobbying = df_lobbying.drop(
['Registrant_Raw', #Same as Registrant
 'Client_raw', # Same as Client
 'Source', # Irrelevant to analysis
 'Typelong' # Same as Type
], axis=1)

# Removing all records from lobbying dataframe without an ID
df_lobbying = df_lobbying.dropna(subset=['UniqID'])

# Capitalizing all inputs in Catcode column
df_lobbying['Catcode'] = df_lobbying['Catcode'].str.upper()

# Replacing missing values in selected boolean columns with 'n' column with 'n'
boolean_columns = ['IsLobbyingFirm','Affiliate','IncludeNSFS','Ind']
df_lobbying[boolean_columns] = df_lobbying[boolean_columns].fillna(value='n')

# Changing datatype of columns
df_lobbying['Amount'] = pd.to_numeric(df_lobbying['Amount'], errors='coerce')
df_lobbying = df_lobbying.dropna(subset=['Amount'])
df_lobbying['Year'] = df_lobbying['Year'].astype('int')
df_lobbying[boolean_columns] = df_lobbying[boolean_columns].apply(lambda col: col.map({'y': True, 'n': False}).astype(bool))

# Dropping all lobbying records with Use == n, as these reports have been amendent/updated and a more recent report is avaliable.  
df_lobbying = df_lobbying[df_lobbying['Use'] != 'n']
df_lobbying = df_lobbying.drop('Use',axis=1)

# Only including records after 2011.
df_lobbying = df_lobbying[df_lobbying['Year'] >= 2011]
df_lobbying = df_lobbying.drop('Affiliate',axis=1) # Dropping the Affiliate column as all records are equal to False after Slicing. 


In [238]:
# Extract quarter dynamically (first character + "Q")
df_lobbying['Quarter'] = 'Q' + df_lobbying['Type'].str.extract(r'(\d)')[0]

# Extract report type based on suffix in 'Type' column
df_lobbying['ReportType'] = df_lobbying['Type'].apply(lambda x: 
    'Standard Report' if len(x) == 2 else
    'Amendment' if x.endswith("a") else
    'Amendment (No Activity)' if x.endswith('an') else
    'No Activity' if x.endswith('n') else
    'Termination' if x.endswith('t') else
    "Termination Amendment" if x.endswith('ta') else
    'Termination (No Activity)' if x.endswith('tn') else
    'Unknown')

df_lobbying.drop('Type',axis=1) # Dropping 'Type' column as it is uneccessary now. 

,UniqID,Registrant,IsLobbyingFirm,Client,UItorg,Amount,Catcode,Self,IncludeNSFS,Ind,Year,Quarter,ReportType
0,82c5f661-a637-45ad-a3a6-b5ba18cf8962,AstraZeneca Pharmaceuticals,False,AstraZeneca Pharmaceuticals,AstraZeneca PLC,1370000.0,H4300,x,False,True,2021,Q4,Amendment
1,84ad3a9e-5864-4227-a802-e268fbf37237,David L Horne LLC,True,Multifamily Lenders Council,Multifamily Lenders Council,15000.0,F4600,n,False,True,2021,Q4,Standard Report
2,85b111b1-5d2e-4107-bc24-0921316e29a5,Echelon Government Affairs,True,Albers Group,Albers Group,10000.0,Y4000,n,False,True,2021,Q4,Standard Report
3,87822a14-12de-478c-a34d-010fa503e539,Western Telecommunications Alliance,False,Western Telecommunications Alliance,Western Telecommunications Alliance,75000.0,C4000,p,False,True,2021,Q4,Standard Report
4,87ff989d-9d12-4fef-84ef-ab69cd616894,Financial Executives International,False,Financial Executives International,Financial Executives International,21650.0,F5000,p,False,True,2021,Q4,Standard Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544068,0854dd96-ed91-4219-99d9-b84bf06ee2f8,American Express,False,American Express,American Express,440000.0,F1400,p,False,True,2022,Q4,Standard Report
1544069,086b65ba-0abe-4ed0-a3e9-a66c9ecc96a3,BL Partners Group,True,CTIA,CTIA,30000.0,C4300,i,False,False,2022,Q4,Standard Report
1544070,b01b51ba-d426-4750-ad00-0ea3c5aa2317,Vectis DC,True,"City of Montebello, CA","City of Montebello, CA",20000.0,X3000,n,False,True,2022,Q4,Standard Report
1544071,b0235c16-0a34-4e06-be62-f42a3bf52d6a,Nickles Group,True,Walmart Inc,Walmart Inc,80000.0,G4300,i,False,False,2022,Q4,Standard Report


### Industry

In [239]:
df_industry = load_csv(lob_indus, save_file = False)
df_industry.columns = ['Client','Sub','Total','Year','Catcode']

Downloading...
From: https://drive.google.com/uc?export=download&id=1yldZYLcZTLIMQ5Xa-e-ecGgQGFWv5n4A
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_indus.csv
100%|██████████| 20.3M/20.3M [00:02<00:00, 9.46MB/s]


In [241]:
# Only including records after 2011.
df_industry = df_industry[df_industry['Year'] >= 2011]

# Capitalizing all inputs in Catcode column
df_industry['Catcode'] = df_industry['Catcode'].str.upper()

#### Catcode References

In [242]:
# Try reading the file with extra handling for irregularities
df_catcode_references = load_csv(CatCodeReference, save_file = False)
df_catcode_references.columns = ['Catcode','Catname','Catorder','Industry','Sector','SectorLong']

Downloading...
From: https://drive.google.com/uc?export=download&id=1vfVA-v-lUMuykF9efl2oDN5DJK2t1PwA
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/CatCodeReference.csv
100%|██████████| 43.0k/43.0k [00:00<00:00, 1.58MB/s]


In [243]:
# merging catcode reference with industry column
df_industry = pd.merge(df_industry, df_catcode_references[['Catcode','Catname', 'Industry', 'Sector']],
                     left_on='Catcode', right_on='Catcode', how='left')

### Agency

In [244]:
df_agency = load_csv(lob_agency, save_file=False)
df_agency.columns = ['UniqID','AgencyID','Agency']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1CFi0Itpi_qg4X4wicg1FIg8S_FJfvCN0
From (redirected): https://drive.google.com/uc?export=download&id=1CFi0Itpi_qg4X4wicg1FIg8S_FJfvCN0&confirm=t&uuid=382167b7-7f79-4eb1-bbc5-0fea547973e7
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_agency.csv
100%|██████████| 256M/256M [00:19<00:00, 12.8MB/s] 


### Issues

In [246]:
df_issue = load_csv(lob_issue, save_file=False)
df_issue.columns = ['SI_ID','UniqID','IssueID','Issue','SpecificIssue','Year']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=18-D_vW4dAHJlvFs4ARK4lgx6KdsAXWoq
From (redirected): https://drive.google.com/uc?export=download&id=18-D_vW4dAHJlvFs4ARK4lgx6KdsAXWoq&confirm=t&uuid=36fb1aae-dec8-40dc-8479-d18f19123652
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_issue.csv
100%|██████████| 787M/787M [01:07<00:00, 11.7MB/s] 


In [247]:
# Only including records after 2011.
df_issue = df_issue[df_issue['Year'] >= 2011]

# Retrieving records with missing values for SpecificIssue 
specific_issue_missing = df_issue[df_issue['SpecificIssue'].isna()]

# Imputing missing values in SpecificIssue column with value from Issue column. 
df_issue['SpecificIssue'] = df_issue['SpecificIssue'].fillna(df_issue['Issue'])

# Dropping IssueID column as this has no value for our analysis. 
df_issue.drop('IssueID',axis=1)

,SI_ID,UniqID,Issue,SpecificIssue,Year
0,3001624,02e92bd6-0159-495e-9d00-8a490a0be8be,Disaster & Emergency Planning,Issues affecting manufacturer of railroad and ...,2022
1,3001625,02e92bd6-0159-495e-9d00-8a490a0be8be,Environment & Superfund,Issues affecting manufacturer of railroad and ...,2022
2,3001626,02e92bd6-0159-495e-9d00-8a490a0be8be,"Labor, Antitrust & Workplace",Issues affecting manufacturer of railroad and ...,2022
3,3001627,02e92bd6-0159-495e-9d00-8a490a0be8be,Roads & Highways,Issues affecting manufacturer of railroad and ...,2022
4,3001628,02e92bd6-0159-495e-9d00-8a490a0be8be,Railroads,Issues affecting manufacturer of railroad and ...,2022
...,...,...,...,...,...
3009859,3076640,0a62db56-bdc4-4f4c-9329-5b420066f53c,Small Business,Women's Business Center program reauthorizatio...,2022
3009860,3076641,0a6de630-f472-4dc2-a796-b58d67831573,Defense,Issues related to tungsten manufacturing,2022
3009861,3076642,0a7f2d03-7d7a-450c-b20a-a48c0fa93e92,Automotive Industry,Issues related to autonomous vehicles Cybersec...,2022
3009862,3076643,0a7f2d03-7d7a-450c-b20a-a48c0fa93e92,Banking,Financial services issues Industrial loan bank...,2022


#### Bills

In [248]:
df_bills = load_csv(lob_bills, save_file=False)
df_bills.columns = ['B_ID', 'SI_ID', 'CongNo', 'Bill_Name']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1qqQvNXtdhyI8KYcctpVUZ3FoIwvjWaG1
From (redirected): https://drive.google.com/uc?export=download&id=1qqQvNXtdhyI8KYcctpVUZ3FoIwvjWaG1&confirm=t&uuid=f866370a-03ea-4032-bd5b-5891778b91f6
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_bills.csv
100%|██████████| 219M/219M [00:19<00:00, 11.0MB/s] 


In [249]:
# Importing Year column from the issues dataframe 
# this is to impute missing values for congress number by cross-referencing Year with a dictionary containing Year and Congress No. at the time. 
df_bills = df_bills.merge(df_issue[['SI_ID', 'Year']], on='SI_ID', how='left')

# Dictionary mapping Congress by year. Retrieved from: https://www.congress.gov/help/field-values/congresses
congress_number_by_year = {
2025: 119.0, 2026: 119.0,2023: 118.0,2024: 118.0,2021: 117.0,2022: 117.0,2019: 116.0,2020: 116.0,
2017: 115.0,2018: 115.0,2015: 114.0,2016: 114.0,2013: 113.0,2014: 113.0,2011: 112.0,
2012: 112.0,2009: 111.0,2010: 111.0,2007: 110.0,2008: 110.0,2005: 109.0,2006: 109.0,
2003: 108.0,2004: 108.0,2001: 107.0,2002: 107.0,1999: 106.0,2000: 106.0,1998: 105.0
}

# Impute missing values
df_bills['CongNo'] = df_bills['CongNo'].fillna(df_bills['Year'].map(congress_number_by_year))

# Dropping 'Year' column again
df_bills = df_bills.drop('Year',axis=1)

# Converting datatype of CongNo column
df_bills['CongNo'] = df_bills['CongNo'].astype('Int64')

In [250]:
# Dropping records with missing value in CongNo, as these records are not connected to Lobby Issues dataframe through Foreign Key - and are hence excessive
df_bills = df_bills.dropna(subset=['CongNo'])

In [251]:
df_bills["Bill_Name"] = df_bills["Bill_Name"].str.replace(r"^H\.?(\d+)$", r"H.R.\1", regex=True)

### Lobbyist

In [252]:
df_lobbyist = load_csv(lob_lobbyist,save_file=False)
df_lobbyist.columns = ['UniqID','Lobbyist_raw','Lobbyist','LobbyistID','Year','OfficialPosition','CID','Formercongmem']

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1q1ZyLvUhsxsbPLdi_CmrNH0FqLoorr6p
From (redirected): https://drive.google.com/uc?export=download&id=1q1ZyLvUhsxsbPLdi_CmrNH0FqLoorr6p&confirm=t&uuid=87d1a784-0cea-463b-a23f-5cf79d54c62c
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/lob_lobbyist.csv
100%|██████████| 410M/410M [00:24<00:00, 16.6MB/s] 


In [253]:
# Only including records after 2011.
df_lobbyist = df_lobbyist[df_lobbyist['Year'] >= 2011]

# Dropping missing values
df_lobbyist = df_lobbyist.dropna(subset=['LobbyistID'])
df_lobbyist = df_lobbyist.dropna(subset=['Year']) # Dropping this record as well as it does not have any UniqId

# Dropping uneccessary column
df_lobbyist = df_lobbyist.drop('Lobbyist_raw',axis=1)

# Chaging data type of Year column
df_lobbyist['Year'] = df_lobbyist['Year'].astype('Int64')

# Imputing missing values in 'Formercongmem' column -- if CID = Blank --> 'n', if CID != Blank --> 'y'
df_lobbyist.loc[df_lobbyist['Formercongmem'].isna(), 'Formercongmem'] = df_lobbyist['CID'].apply(
    lambda x: 'n' if pd.isna(x) else 'y')

# Changing data type of Formercongmem column to Boolean
df_lobbyist['Formercongmem'] = df_lobbyist['Formercongmem'].map({'y': True, 'n': False}).astype(bool)

# Imputing missing values for CID column
df_lobbyist['CID'] = df_lobbyist['CID'].fillna(value='Not Congress Member')


In [254]:
# Pringint unique values in official position column
print(len(df_lobbyist['OfficialPosition'].unique()))

# Considering the amount of missing values and unique values - OfficialPosition column is dropped due to providing small value to further analysis
df_lobbyist = df_lobbyist.drop('OfficialPosition',axis=1)

41623


#### Candidate IDs

In [255]:
df_candidate = load_csv(CandidateIds,save_file=False)
df_candidate.columns = ['CID', 'CRPName', 'Party', 'DistIDRunFor', 'FECCandID']

Downloading...
From: https://drive.google.com/uc?export=download&id=1k2lhAcuSQbn5z7NN3USPEbX_Rg5VV_Ky
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/CandidateIds.csv
100%|██████████| 970k/970k [00:01<00:00, 681kB/s]


In [256]:
# Merge on the common 'CID' column to include the columns from df_candidate
df_lobbyist = pd.merge(
    df_lobbyist,
    df_candidate[['CID', 'CRPName', 'Party', 'DistIDRunFor', 'FECCandID']],
    on='CID',
    how='left'
)

## Narrowing Scope

In [257]:
# Only including lobbying reports related to specific issues
chosen_issues = ['Health Issues', 'Medical Research & Clin Labs', 'Pharmacy', 'Medicare & Medicaid', 'Science & Technology', 'Taxes']
df_issue = df_issue[df_issue['Issue'].isin(chosen_issues)]

In [258]:
# Dropping lobbying reports that are no longer referenced in df_issue table
df_lobbying = df_lobbying[df_lobbying['UniqID'].isin(df_issue['UniqID'])]

# dropping lobbyist records for lobbyists that are no longer involved in the lobbying reports from cleaned df_lobbying df.
df_lobbyist = df_lobbyist[df_lobbyist['UniqID'].isin(df_lobbying['UniqID'])]

# droppping bills records that are not related to any of the chosen issues. 
df_bills = df_bills[df_bills['SI_ID'].isin(df_issue['SI_ID'])]

# droppping agency records that are not related to any of the current reports. 
df_agency = df_agency[df_agency['UniqID'].isin(df_lobbying['UniqID'])]

# droppping industry records that are not related to any of the current reports.
df_industry = df_industry[df_industry['Catcode'].isin(df_lobbying['Catcode'])]


#### Importing additional information about bills

In [259]:
df_congress_bill_data = load_csv(congress_bill_data_id, save_file=False)
df_congress_bill_data.columns = ['bill_number','congress_number','bill_title','policy_area','introduced_date',
                                 'bill_type','full_bill_code','sponsor_name','sponsor_party','sponsor_bioguide_id']
df_congress_bill_data = df_congress_bill_data.iloc[1:] # dropping first row of dataframe

Downloading...
From: https://drive.google.com/uc?export=download&id=1-MnzV0iZ2FPYiuQgOmggjmXvxGmBzYV8
To: /Users/kristianmadslangrud/Documents/GitHub/master_thesis25/congress_bill_data.csv
100%|██████████| 4.26M/4.26M [00:00<00:00, 7.13MB/s]


In [260]:
df_congress_bill_data['congress_number'] = df_congress_bill_data['congress_number'].astype('Int64')
df_congress_bill_data = df_congress_bill_data.merge(
    df_bills[['B_ID', 'CongNo', 'Bill_Name']],
    left_on=['full_bill_code', 'congress_number'],
    right_on=['Bill_Name', 'CongNo'],
    how='left'
).drop(columns=['Bill_Name', 'CongNo'],axis = 1)  # Drop duplicate columns after merge if needed

In [261]:
# Dropping bill records we don't have any data sponsor data on. 
df_bills = df_bills[df_bills['B_ID'].isin(df_congress_bill_data['B_ID'])]

## Restructuring Data for Efficiency

#### Main Table

In [ ]:
# Merge all tables on 'UniqID' to capture all required fields
df_master_table = (
    df_lobbying
    .merge(df_agency, on='UniqID', how='left')
    .merge(df_lobbyist, on='UniqID', how='left')
    .merge(df_issue, on='UniqID', how='left')
)

# Select only the required unique columns
df_master_table = df_master_table[['UniqID', 'AgencyID', 'LobbyistID', 'SI_ID']].drop_duplicates()

# Add a unique MasterID
df_master_table.insert(0, 'MasterID', range(1001, 1001 + len(df_master_table)))

df_master_table['AgencyID'] = df_master_table['AgencyID'].astype('Int64') # changing data type of AgencyID column

In [263]:
df_master_table.isnull().sum()

MasterID          0
UniqID            0
AgencyID      10211
LobbyistID      976
SI_ID             0
dtype: int64

#### Industry Table

In [264]:
#Summarize lobbying data to only include Ind=True records
df_filtered_industry_summary = (
    df_lobbying[df_lobbying["Ind"] == True]
    .groupby(["Catcode", "Year"])
    .agg(
        Total_Lobbying_Reported=("Amount", "sum"),  # Sum lobbying spending that counts towards industry totals
    ).reset_index())

# Summarize industry data by Catcode + Year (CPK in this table)
df_industry_sum = (
    df_industry.groupby(["Catcode", "Year"])
    .agg(
        Total_Lobbying_Spending=("Total", "sum"),  # Sum total spending per industry per year
        Unique_Clients=("Client", "nunique")  # Count unique clients per industry per year
    ).reset_index()
)

# Merge with industry summary (which contains total lobbying spending and unique clients)
df_industry_summary = df_filtered_industry_summary.merge(
    df_industry_sum,
    on=["Catcode", "Year"],
    how="left")

#### Agency Table

In [265]:
# Dropping duplicate entries of AgencyID and Agency combination
df_agency_final = df_agency[['AgencyID', 'Agency']].drop_duplicates()

# Displaying potential duplicates in AgencyID column
df_agency_final[df_agency_final.duplicated(subset=['AgencyID'], keep=False)].sort_values('AgencyID')

# due to some inconsistencies in the typing of Agency Name, we have a few duplicated Agency ID's. 
to_drop = [
    'Federal Deposit Insurance Commission',
    'Postal Rate Commission',
    'Defense Security Assistance Agency',
    'ACCESS Board',
    'Office of National Aids Policy'
]
df_agency_final = df_agency_final[~df_agency_final['Agency'].isin(to_drop)]


#### Lobbyist Table

In [266]:
df_lobbyist_final = df_lobbyist[['LobbyistID', 'Lobbyist', 'Formercongmem', 'Party']].drop_duplicates()

In [267]:
# Imputing missing values for political party for former congress-men
missing_political_party = {
    'Y0000022381L' : 'R','Y0000014172L' : 'R','Y0000041393L' : 'R','Y0000041198L' : 'D','Y0000031994L' : 'R','Y0000040464L' : 'R',
    'Y0000041214L' : 'D','Y0000040343L' : 'R','Y0000026610L' : 'R','Y0000042045L' : 'R','Y0000041638L' : 'R','Y0000027785L' : 'R',
    'Y0000041137L' : 'D','Y0000009728L' : 'R','Y0000041060L' : 'R','Y0000027979L' : 'R','Y0000040107L' : 'D','Y0000030165L' : 'D',
    'Y0000008316L' : 'R','Y0000041693L' : 'D','Y0000041437L' : 'D','Y0000000158L' : 'R','Y0000040464M' : 'R','Y0000040575L' : 'R',
    'Y0000042027L' : 'D','Y0000009666L' : 'D','Y0000038683L' : 'R','Y0000046534L' : 'R','Y0000041860L' : 'D','Y0000028461L' : 'R',
    'Y0000014475L' : 'D','Y0000017606L' : 'D','Y0000011321L' : 'R','Y0000042041L' : 'R','Y0000041375L' : 'D','Y0000041586L' : '3',
    'Y0000041364L' : 'D','Y0000017415L' : 'R','Y0000008436L' : 'R','Y0000045198L' : 'R','Y0000006265L' : 'R','Y0000035408L' : 'R',
    'Y0000040391L' : 'R','Y0000041827L' : 'R','Y0000041317L' : 'R','Y0000042035L' : 'D','Y0000040953L' : 'D','Y0000012587L' : 'D',
    'Y0000000545L' : 'D','Y0000005168L' : 'D','Y0000005429L' : 'R','Y0000028048L' : 'R','Y0000039653L' : 'D','Y0000015675L' : 'D',
    'Y0000014080L' : 'D','Y0000041094L' : 'D','Y0000025402L' : 'D','Y0000033912L' : 'R','Y0000027950L' : 'R','Y0000003201L' : 'D',
    'Y0000046532L' : 'D','Y0000027397L' : 'D','Y0000040428L' : 'R','Y0000017752L' : 'D','Y0000000969L' : 'D','Y0000016284L' : 'R',
    'Y0000001540L' : 'D','Y0000008788L' : 'D','Y0000055101L' : 'D','Y0000019350L' : 'R','Y0000028862L' : 'D','Y0000047746L' : 'D',
    'Y0000020586L' : 'D','Y0000007555L' : 'R'
    }

df_lobbyist_final['Party'] = df_lobbyist_final['Party'].fillna(df_lobbyist_final['LobbyistID'].map(missing_political_party))


#### Issue Table

In [ ]:
# Only keeping necessary columns for new table
df_issue_final = df_issue[['SI_ID','Issue','SpecificIssue','Year']]

df_issue_final = df_issue_final.merge(
    df_bills[['SI_ID','B_ID']],
    on = 'SI_ID',
    how = 'left'
)

#### Bills Table

In [ ]:
df_bills_final = df_bills[['B_ID','CongNo','Bill_Name']] # only keeping necessary columns
df_bills_final = df_bills_final[['B_ID', 'CongNo', 'Bill_Name']].drop_duplicates() # Dropping duplicates from dataframe. 

#### Sponsor Table

In [297]:
# Only keeping necessary columns
df_sponsor = df_congress_bill_data[['sponsor_bioguide_id','sponsor_name','sponsor_party','B_ID']]

In [301]:
df_sponsor[df_sponsor['sponsor_bioguide_id'].isna()]

,sponsor_bioguide_id,sponsor_name,sponsor_party,B_ID
1463326,NaN,NaN,NaN,hjres45-111
1543604,NaN,NaN,NaN,ZZ7020917
1543605,NaN,NaN,NaN,ZZ7021646
1543606,NaN,NaN,NaN,ZZ7022752
1543607,NaN,NaN,NaN,ZZ7027776
...,...,...,...,...
1543693,NaN,NaN,NaN,ZZ7190094
1607083,NaN,NaN,NaN,ZZ6846510
1607132,NaN,NaN,NaN,ZZ6847579
1607133,NaN,NaN,NaN,ZZ6889222
